In [1]:
import pandas as pd
import logging 
import numpy as np
import re 
from IPython.display import display
from sqlalchemy import MetaData, Table
from database_utils import DatabaseConnector
from data_extraction import DataExtractor

In [ ]:
# from 31 july 
# try to get to the bottom of the 'lat' thing, I have a list of 10 but am confused about how to take them out 

In [2]:
# LOGGING 
print('Started cleaning_store_details')

#creating instance of dataextractor 
instance = DataExtractor()

#retrieving the data from the stores API - this takes around 90 seconds 
df = instance.retrieve_stores_data()  

Started cleaning_store_details
retrieve_stores_data is working
read_api_key is working
read_api_key is done
Skipped Stores DataFrame:
Empty DataFrame
Columns: []
Index: []


In [3]:
#making a back up of the dataframe 
df_backup = df.copy()

In [70]:
# run this to reset the df 
df = df_backup.copy()

In [52]:
# Locate the row(s) where 'lat' is 'N/A'
na_rows = df.loc[df['lat'] == 'N/A']

null_rows = df.loc[df['lat'] == 'NULL']


# Print the 'N/A' row(s) for visual inspection
print("Rows where 'lat' is 'N/A':")
display(na_rows)
display(null_rows)

Rows where 'lat' is 'N/A':


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
217,217,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
405,405,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
437,437,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [58]:
# Define the regex pattern to match invalid latitude values
pattern = r'^[A-Za-z0-9]+$'

# Create a boolean mask for rows to keep: rows that do not match the pattern and are not 'NULL' or 'N/A'
mask = ~df['lat'].str.contains(pattern, na=False) & ~df['lat'].isin(['NULL'])

# Filter the DataFrame using the mask
df = df[mask]

df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 441 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          441 non-null    int64 
 1   address        441 non-null    object
 2   longitude      441 non-null    object
 3   lat            1 non-null      object
 4   locality       441 non-null    object
 5   store_code     441 non-null    object
 6   staff_numbers  441 non-null    object
 7   opening_date   441 non-null    object
 8   store_type     441 non-null    object
 9   latitude       440 non-null    object
 10  country_code   441 non-null    object
 11  continent      441 non-null    object
dtypes: int64(1), object(11)
memory usage: 44.8+ KB


In [92]:
df = df_backup.copy()

num_rows = df.shape[0]
print(f"Number of rows store data before cleaning: {num_rows}")

# remove garbage records and NULL from lat 

# Define the regex pattern to match invalid latitude values
pattern = r'^[A-Za-z0-9]+$'

# Create a boolean mask for rows to keep: rows that do not match the pattern and are not 'NULL' or 'N/A'
mask = ~df['lat'].str.contains(pattern, na=False) & ~df['lat'].isin(['NULL'])

# Filter the DataFrame using the mask
df = df[mask]

num_rows = df.shape[0]
print(f"Number of rows store data after dropping lat column: {num_rows}")

# replacing incorrect spellings of continents 
continent_replacements = {
    'eeEurope': 'Europe',
    'eeAmerica': 'America'
}

df['continent'] = df['continent'].replace(continent_replacements)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning continent: {num_rows}")


num_rows = df.shape[0]
print(f"Number of rows store data after cleaning: {num_rows}")
#returning the dataframe 


Number of rows store data before cleaning: 451
Number of rows store data after dropping lat column: 441
Number of rows store data after cleaning continent: 441
Number of rows store data after cleaning: 441


In [44]:
#sandbox for 'lat'

# print df
print(df.info())

# create empty list for lat_items 
str_items = []
nonetype_items = []
empty_items = [] 
lat_items = []
null_str_items = [] 

# Go through the series 'lat' and append each item to lat_items
for x in df['lat']:
    if x is None: 
        #print('Nonetype:')
        nonetype_items.append(x)
    elif x == 'None':  # Check for string 'None'
        #print('string')
        str_items.append(x)
    elif x == 'NULL':
        #print('string')
        null_str_items.append(x)
    elif x == '':  # Check for empty string
        #print('empty string:')
        empty_items.append(x)
    else:
        #print('dont know')
        lat_items.append(x)

# Print out lat_items
print('lat', lat_items)
print('str:', str_items) 
print('nonetype', nonetype_items)
print('empty', empty_items)
print('null string:', null_str_items)

# drop na rows from the 'lat' column 
#df = df.dropna(subset=['lat'])

# # Define the regex pattern
# string_pattern = r'^[A-Za-z0-9]+$'

# # Fill NaN values with a placeholder that does not match the regex
# df['lat'] = df['lat'].fillna('@@@')

# # Filter rows where 'lat' matches the pattern
# df = df[df['lat'].str.match(string_pattern)]

# # Reset 'lat' column to NaN where the placeholder was used
# df['lat'] = df['lat'].replace('placeholder', np.nan)

# Go through the series 'lat' and append each item to lat_items
for x in df['lat']:
    if x is None: 
        #print('Nonetype:')
        nonetype_items.append(x)
    elif x == 'None':  # Check for string 'None'
        #print('string')
        str_items.append(x)
    elif x == 'NULL':
        #print('string')
        null_str_items.append(x)
    elif x == '':  # Check for empty string
        #print('empty string:')
        empty_items.append(x)
    else:
        #print('dont know')
        lat_items.append(x)

# Print out lat_items
print('lat', lat_items)
print('str:', str_items) 
print('nonetype', nonetype_items)
print('empty', empty_items)
print('null string:', null_str_items)


# go through the remaining lat items and add them to new_lat_items
# new_lat_items = [] 
# for x in df['lat']:
#     new_lat_items.append(x)

# print new lat items 
# print(new_lat_items)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          451 non-null    int64 
 1   address        451 non-null    object
 2   longitude      451 non-null    object
 3   lat            11 non-null     object
 4   locality       451 non-null    object
 5   store_code     451 non-null    object
 6   staff_numbers  451 non-null    object
 7   opening_date   451 non-null    object
 8   store_type     451 non-null    object
 9   latitude       450 non-null    object
 10  country_code   451 non-null    object
 11  continent      451 non-null    object
dtypes: int64(1), object(11)
memory usage: 42.4+ KB
None
lat ['N/A', '13KJZ890JH', '2XE1OWOC23', 'OXVE5QR07O', 'VKA5I8H32X', 'LACCWDI0SB', 'A3O5CBWAMD', 'UXMWDMX1LC']
str: []
nonetype [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 63 to 447
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          10 non-null     int64 
 1   address        10 non-null     object
 2   longitude      10 non-null     object
 3   lat            10 non-null     object
 4   locality       10 non-null     object
 5   store_code     10 non-null     object
 6   staff_numbers  10 non-null     object
 7   opening_date   10 non-null     object
 8   store_type     10 non-null     object
 9   latitude       10 non-null     object
 10  country_code   10 non-null     object
 11  continent      10 non-null     object
dtypes: int64(1), object(11)
memory usage: 1.0+ KB


In [25]:
#sandbox for latitde 
 
#df.info()
num_rows = df.shape[0]
print(f"Number of rows store data before cleaning: {num_rows}")

df['latitude']

print(type(df['latitude']))

# Create an empty list to store non-matching card numbers
non_matching = []

# Define a regular expression pattern for numbers only
pattern = r'^-?\d+(\.\d+)?$'

# Ensure all elements in the 'card_number' column are strings
df['latitude'] = df['latitude'].astype(str)

for x in df['latitude']:
    if not re.match(pattern, str(x)):
        non_matching.append(x)

# Print the list of non-matching card numbers
print("List of non-matching card numbers:", non_matching)
print(len(non_matching))

df = df[df['latitude'].apply(lambda x: bool(re.match(pattern, x)))]


# df = df.dropna(subset=['latitude'])
print(df.info())
num_rows = df.shape[0]
print(f"Number of rows store data after cleaning latitide column: {num_rows}")

Number of rows store data before cleaning: 451
<class 'pandas.core.series.Series'>
List of non-matching card numbers: ['None', '1CJ5OAU4BR', '3ZR3F89D97', 'NULL', 'L13EQEQODP', 'J3BPB68Z1J', 'AJHOMDOHZ4', 'NULL', 'AE7EEW4HSS', 'NULL', 'JZP8MIJTPZ']
11
<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 1 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          440 non-null    int64 
 1   address        440 non-null    object
 2   longitude      440 non-null    object
 3   lat            0 non-null      object
 4   locality       440 non-null    object
 5   store_code     440 non-null    object
 6   staff_numbers  440 non-null    object
 7   opening_date   440 non-null    object
 8   store_type     440 non-null    object
 9   latitude       440 non-null    object
 10  country_code   440 non-null    object
 11  continent      440 non-null    object
dtypes: int64(1), object(11)
memory usage: 44.

In [10]:
# code to clean the dataframe 
# Number of rows store data before cleaning: 451
# Number of rows store data after dropping lat and cleaning latitide column: 440
# Number of rows store data after cleaning locality: 440
# Number of rows store data after cleaning continent: 440
# Number of rows store data after cleaning opening date: 428
# Number of rows store data after cleaning: 428

df.info()
num_rows = df.shape[0]
print(f"Number of rows store data before cleaning: {num_rows}")

# Clean the latitude column


df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

df = df.dropna(subset=['latitude'])

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning latitide column: {num_rows}")

# Dropping the 'latitude' column as it's empty 
df = df.drop(columns=['lat'])

num_rows = df.shape[0]
print(f"Number of rows store data after dropping lat column: {num_rows}")

# filtering out items in locality that aren't real place names or NULL 
pattern = r'^[a-zA-Z\s-]+$'
df = df[df['locality'].str.match(pattern)]
df['locality'] = df['locality'].replace('NULL', np.nan)
df = df.dropna(subset=['locality'])

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning locality: {num_rows}")

# replacing incorrect spellings of continents 
continent_replacements = {
    'eeEurope': 'Europe',
    'eeAmerica': 'America'
}

df['continent'] = df['continent'].replace(continent_replacements)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning continent: {num_rows}")

# LOGGING: checking everything has worked 
#print(df['continent'].unique())
#print(df['store_type'].unique()) 
#print(df['country_code'].unique()) 
#print(df['continent'].unique())

# LOGGING: checking of datetime before is datetime64 datetype  
#is_datetime_before = pd.api.types.is_datetime64_any_dtype(df['opening_date'])

# converting opening date to datetime object 
df['opening_date'] = pd.to_datetime(df['opening_date'], format='%Y-%m-%d', errors='coerce')

# dropping any rows which contain missing values  
df = df.dropna(axis=0)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning opening date: {num_rows}")

# LOGGING: checking the conversion worked 
#is_datetime_after = pd.api.types.is_datetime64_any_dtype(df['opening_date'])
#print(f"Is 'dates' column datetime64 dtype? {is_datetime_before}")
#print(f"Is 'dates' column datetime64 dtype? {is_datetime_after}")

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning: {num_rows}")
#returning the dataframe 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          451 non-null    int64 
 1   address        451 non-null    object
 2   longitude      451 non-null    object
 3   lat            11 non-null     object
 4   locality       451 non-null    object
 5   store_code     451 non-null    object
 6   staff_numbers  451 non-null    object
 7   opening_date   451 non-null    object
 8   store_type     451 non-null    object
 9   latitude       450 non-null    object
 10  country_code   451 non-null    object
 11  continent      451 non-null    object
dtypes: int64(1), object(11)
memory usage: 42.4+ KB
Number of rows store data before cleaning: 451
Number of rows store data after cleaning latitide column: 440
Number of rows store data after dropping lat column: 440
Number of rows store data after cleaning locality: 440
Number of rows store

In [ ]:

non_matching_cards = []
strings = []
numbers = []

for x in df['card_number']:
    if isinstance(x, str):
        strings.append(x)
    elif isinstance(x, int):
        numbers.append(x)
    else:
        non_matching_cards.append(x)